# Read in e3t and create a +/-2 m SSH versions 

In [132]:
import xarray as xr
import numpy as np
import time
from datetime import datetime, timedelta
from dateutil.parser import parse
import os
from netCDF4 import Dataset

## User input

In [135]:
date_begin = parse('5 june 2015')
date_end   = parse('12 june 2015')
path       = '/results2/SalishSea/nowcast-green.201806/'
filetype   = 'carp_T'
depth_change = 2
out_e3t_frac = '/home/rmueller/Projects/MIDOSS/analysis-rachael/notebooks/vvl/e3t_frac_dz_2.nc'

# pick Salmon Bank location [256,265], but remember that MOHID is transposed! such that SSC [yloc_ssc,xloc_ssc]-> [xloc_ssc,yloc_ssc]_mohid = [yloc_mohid,xloc_mohid]
yloc_mohid       = 265
xloc_mohid       = 256

## Ashu's function to transpose matrices to MOHID orientation

In [67]:
def mung_array(SSC_gridded_array, array_slice_type):
    """Transform an array containing SalishSeaCast-gridded data and transform it 
       into a MOHID-gridded array by:
         1) Cutting off the grid edges
         2) Transposing the X and Y axes
         3) Flipping the depth dimension, if it is present
         4) Converting the NaNs to 0
    
        :arg SSC_gridded_array: SalishSeaCast-gridded array
        :type numpy.ndarray: :py:class:'ndarray'
    
        :arg array_slice_type: str, one of '2D' or '3D'
        :type str: :py:class:'str'

        :return MOHID_gridded_array: MOHID-gridded array produced by applying operation
                                     1-4 on SSC_gridded_array
        :type numpy.ndarray: :py:class:'ndarray'
    """
    shape = SSC_gridded_array.shape
    ndims = len(shape)
    assert(array_slice_type in  ('2D', '3D')), f"Invalid option {array_slice_type}. array_slice_type must be one of ('2D', '3D')"
    if array_slice_type is '2D':
        assert(ndims in (2,3)), f'The shape of the array given is {shape}, while the option chosen was {array_slice_type}'
        if ndims == 2:
            MOHID_gridded_array = SSC_gridded_array[1:897:,1:397]
            del(SSC_gridded_array)
            MOHID_gridded_array = np.transpose(MOHID_gridded_array, [1,0])
        else:
            MOHID_gridded_array = SSC_gridded_array[...,1:897:,1:397]
            del(SSC_gridded_array)
            MOHID_gridded_array = np.transpose(MOHID_gridded_array, [0,2,1])

    else:
        assert(ndims in (3,4)), f'The shape of the array given is {shape}, while the option chosen was {array_slice_type}'
        MOHID_gridded_array = SSC_gridded_array[...,1:897:,1:397]
        del(SSC_gridded_array)
        if ndims == 3:
            MOHID_gridded_array = np.transpose(MOHID_gridded_array, [0,2,1])
            MOHID_gridded_array = np.flip(MOHID_gridded_array, axis = 0)
        else:
            MOHID_gridded_array = np.transpose(MOHID_gridded_array, [0,1,3,2])
            MOHID_gridded_array = np.flip(MOHID_gridded_array, axis = 1)

    MOHID_gridded_array = np.nan_to_num(MOHID_gridded_array).astype('float64')

    return MOHID_gridded_array

## Ashu's function for writing HDF5 file

In [128]:
def write_grid(data, datearrays, metadata, filename, groupname, accumulator, compression_level):
    shape = data[0].shape
    with h5py.File(filename) as f:
        time_group = f.get('/Time')
        if time_group is None:
            time_group = f.create_group('/Time')
        data_group_path = f'/Results/{groupname}'
        data_group = f.get(data_group_path)
        if data_group is None:
            data_group = f.create_group(data_group_path)

        for i, datearray in enumerate(datearrays):
            numeric_attribute = ((5 - len(str(i + accumulator))) * '0') + str(i + accumulator)
            child_name = 'Time_' + numeric_attribute
            timestamp = time_group.get(child_name)
            if timestamp is None:
                dataset = time_group.create_dataset(
                    child_name,
                    shape = (6,),
                    data = datearray,
                    chunks = (6,),
                    compression = 'gzip',
                    compression_opts = compression_level
                    )
                time_metadata = {
                    'Maximum' : np.array(datearray[0]),
                    'Minimum' : np.array([-0.]),
                    'Units' : b'YYYY/MM/DD HH:MM:SS'
                    }
                dataset.attrs.update(time_metadata)
            else:
                assert (np.asarray(timestamp) == datearray).all(), f'Time record {child_name} exists and does not match with {datearray}'

            child_name = groupname + '_' + numeric_attribute
            if data_group.get(child_name) is not None:
                print(f'Dataset already exists at {child_name}')
            else:
                dataset = data_group.create_dataset(
                    child_name,
                    shape = shape,
                    data = data[i],
                    chunks = shape,
                    compression = 'gzip',
                    compression_opts = compression_level
                    )
                dataset.attrs.update(metadata)

## Generate list of dates from user input

In [37]:
daterange = [date_begin, date_end]
# append all filename strings within daterange to lists
e3t_list = []
for day in range(np.diff(daterange)[0].days + 1):
    datestamp = daterange[0] + timedelta(days = day)
    datestr1 = datestamp.strftime('%d%b%y').lower()
    datestr2 = datestamp.strftime('%Y%m%d')

    # check if file exists. exit if it does not. add path to list if it does.
    file_path = f'{path}{datestr1}/SalishSea_1h_{datestr2}_{datestr2}_{filetype}.nc'
    if not os.path.exists(file_path):
        print(f'File {file_path} not found. Check Directory and/or Date Range.')
    e3t_list.append(file_path)

In [38]:
e3t_list

['/results2/SalishSea/nowcast-green.201806/05jun15/SalishSea_1h_20150605_20150605_carp_T.nc',
 '/results2/SalishSea/nowcast-green.201806/06jun15/SalishSea_1h_20150606_20150606_carp_T.nc',
 '/results2/SalishSea/nowcast-green.201806/07jun15/SalishSea_1h_20150607_20150607_carp_T.nc',
 '/results2/SalishSea/nowcast-green.201806/08jun15/SalishSea_1h_20150608_20150608_carp_T.nc',
 '/results2/SalishSea/nowcast-green.201806/09jun15/SalishSea_1h_20150609_20150609_carp_T.nc',
 '/results2/SalishSea/nowcast-green.201806/10jun15/SalishSea_1h_20150610_20150610_carp_T.nc',
 '/results2/SalishSea/nowcast-green.201806/11jun15/SalishSea_1h_20150611_20150611_carp_T.nc',
 '/results2/SalishSea/nowcast-green.201806/12jun15/SalishSea_1h_20150612_20150612_carp_T.nc']

### Create mask

In [68]:
mask = mung_array(xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSn3DMeshMaskV17-02').isel(time = 0).tmask.values, '3D')

In [81]:
mask.shape

(40, 396, 896)

### Test process with one file

In [92]:
data = xr.open_dataset(e3t_list[0])
datetimelist = data.time_counter.values.astype('datetime64[s]').astype(datetime)
datearrays = [np.array(
        [d.year, d.month, d.day, d.hour, d.minute,d.second]
        ).astype('float64') for d in datetimelist]
del(datetimelist)
e3t = data.e3t.values
e3t = mung_array(e3t, '3D')
e3t = e3t*mask
metadata = {
   'FillValue' : np.array([0.]),
   'Units' : b'?C'
   }

In [94]:
e3t.shape

(24, 40, 396, 896)

In [103]:
total_depth = e3t.sum(1)
total_depth.shape

(24, 396, 896)

In [113]:
tmp = total_depth[1,1,1]
if tmp != 0:
    print("non-zero")

In [107]:
frac = np.empty_like(e3t)
for z in range(e3t.shape[1]):
    frac[1,z,256,265] = e3t[1,z,256,265]/total_depth[1,256,265]

In [116]:
frac[1,:,256,265]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.18629024,  0.18699895,
        0.15091983,  0.11079468,  0.07471557,  0.04797064,  0.03084826,
        0.02089261,  0.0154253 ,  0.01251671,  0.01099547,  0.01020691,
        0.00980005,  0.00959063,  0.00948297,  0.00942766,  0.00939925,
        0.00938466,  0.00937717,  0.00937332,  0.00937135,  0.00937034,
        0.00936982,  0.00936955,  0.00936941,  0.00936934,  0.0093693 ])

In [119]:
e3t_new = total_depth[1,256,265] * frac

In [120]:
e3t_new.shape

(24, 40, 396, 896)

In [121]:
print(e3t_new[1,:,256,265] - e3t[1,:,256,265])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.]


#### Now try with adding 2m

In [124]:
for z in range(e3t.shape[1]):
    frac[1,z,256,265] = e3t[1,z,256,265]/(total_depth[1,256,265])

e3t_new = (total_depth[1,256,265] + 2) * frac
# print difference between e3t adjusted up by 2m and original e3t
print(e3t_new[1,:,256,265] - e3t[1,:,256,265])

[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.37258048  0.37399789  0.30183966  0.22158936  0.14943115  0.09594128
  0.06169653  0.04178523  0.03085061  0.02503342  0.02199094  0.02041383
  0.0196001   0.01918126  0.01896594  0.01885531  0.0187985   0.01876932
  0.01875434  0.01874665  0.0187427   0.01874067  0.01873963  0.0187391
  0.01873882  0.01873868  0.01873861]


#### now loop through time and space to get scale values for all locations and times

In [127]:
total_depth = e3t.sum(1)
e3t_frac_dz = np.empty_like(e3t)

for t in range(e3t.shape[0]):
    for i in range(e3t.shape[2]):
        for j in range(e3t.shape[3]):
            for z in range(e3t.shape[1]):
                e3t_frac_dz[t,z,i,j] = e3t[t,z,i,j]/total_depth[t,i,j]

/home/rmueller/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


In [136]:
print('saving to ', out_e3t_frac)

saving to  /home/rmueller/Projects/MIDOSS/analysis-rachael/notebooks/vvl/e3t_frac_dz_2.nc


In [142]:
xrfrac = xr.DataArray(e3t_frac_dz)
xrfrac.to_netcdf('test_e3t_frac.nc')

In [137]:
ncout = Dataset(out_e3t_frac,'w','NETCDF4');
ncout.createDimension('lon', e3t.shape[3]); # I'm not sure this is right
ncout.createDimension('lat', e3t.shape[2]);  # lat/lon might be swapped
ncout.createDimension('depth', e3t.shape[1])
ncout.createDimension('time', e3t.shape[0]);
e3tvar = ncout.createVariable('e3t','float32',('time','depth','lat','lon')); e3tvar.setncattr('units','mm'); e3tvar[:] = e3t_frac_dz;
ncout.close();
#e3t_frac_dz.to_netcdf(path=out_e3t_frac)

PermissionError: [Errno 13] Permission denied: b'/home/rmueller/Projects/MIDOSS/analysis-rachael/notebooks/vvl/e3t_frac_dz_2.nc'

In [ ]:
# Calculate new e3t based on desired depth change
e3t_new = (total_depth[1,256,265] + depth_change) * e3t_frac_dz

## Scale process to calculate new e3t for all days in run

In [89]:
# function call: process_grid(e3t_list, 'e3t', dirname+e3t, 'vvl', compression_level)
# def process_grid(file_paths, datatype, filename, groupname, compression_level, weighting_matrix_obj=None)
for file_path in e3t_list:
    data = xr.open_dataset(file_path)
    datetimelist = data.time_counter.values.astype('datetime64[s]').astype(datetime)
    datearrays = [np.array(
        [d.year, d.month, d.day, d.hour, d.minute,d.second]
        ).astype('float64') for d in datetimelist]
    del(datetimelist)
    data = data.e3t.values
    e3t = mung_array(data, '3D')
    e3t = e3t*mask
    metadata = {
       'FillValue' : np.array([0.]),
       'Units' : b'?C'
       }
    e3t_new = (total_depth[1,256,265] + depth_change) * e3t_frac_dz
    
    # need to fill out datearrays, metadata, filename, groupname, accumulator, compression_level
    write_grid(e3t_new, datearrays, metadata, filename, groupname, accumulator, compression_level)

In [57]:
data.shape


(24, 40, 898, 398)

## load mask

In [17]:
#3D mask truncated and transposed at needed by MOHID
mask_3D = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSn3DMeshMaskV17-02').tmask.isel(time=0).values[...,1:897, 1:397]
mask_3D = np.transpose(mask_3D, [0,2,1])

In [14]:
e3t_list = forcing_paths.salishseacast_paths(date_begin, date_end, salishseacast_path, 'carp_T')

datetime.datetime(2015, 6, 5, 0, 0)